# **HW 1**

In [ ]:
from re import L
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import pandas as pd
import csv

In [ ]:
xyz_curves = "/content/cieXYZ_curves.csv"
csvfile = pd.read_csv(xyz_curves)
print(csvfile)

# print(csvfile.iloc[:, 0])

lambda_x = csvfile.iloc[:, 1]

for value in lambda_x:
    print(value)

## **Step 1**

In [ ]:
h = 6.62606957e-34
c = 299792458
k_B = 1.3806488e-23

temps = np.linspace(1000, 30000, 291)

lambda_nm = csvfile.iloc[:, 0]
lambda_x = csvfile.iloc[:, 1]
lambda_y = csvfile.iloc[:, 2]
lambda_z = csvfile.iloc[:, 3]

# Μετατροπή nm -> m
lambda_m = lambda_nm.to_numpy() * 1e-9

def planck_radiance(lambda_m, T):
    return 2*h*c**2 / ((lambda_m**5) * (np.exp((h*c)/(lambda_m*k_B*T)) - 1))  # B_T(λ) for every λ

# Υπολογισμός (X, Y, Z) με διακριτό άθροισμα
def XYZ_calc(T):
    B = planck_radiance(lambda_m, T)
    step_of_L_m = lambda_nm[1]-lambda_nm[0]
    X = np.sum(B * lambda_x)*step_of_L_m
    Y = np.sum(B * lambda_y)*step_of_L_m
    Z = np.sum(B * lambda_z)*step_of_L_m
    return X, Y, Z

## **Step 2**

In [ ]:
Curves_x_y = []
curves_x = []
curves_y = []
curves_T = []
for T in temps:
    X, Y, Z = XYZ_calc(T)
    # print(f"for T = {T} K, the X = {X}, Y = {Y}, Z = {Z}")

    # Κανονικοποίηση σε (x, y)
    s = (X + Y + Z)
    x = X / s
    y = Y / s

    print(f"for T = {T} K, the x = {x}, y = {y}")

    # c = round(1000*x)
    # r = round(1000*y)
    # Curves_x_y.append([c, r])

    curves_x.append(round(1000*x))
    curves_y.append(round(1000*y))
    curves_T.append(T)

## **Step 3**

### 3.1 Using Colour

In [ ]:
!pip install colour-science colour-demosaicing
!pip install colour-demosaicing

import colour

from colour_demosaicing import (
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_Menon2007,
    mosaicing_CFA_Bayer)

colour.plotting.colour_style();
colour.utilities.describe_environment();

In [ ]:
CFA = colour.io.read_image('CFA_stream.png')

colour.plotting.plot_image(
    colour.cctf_encoding(CFA))


colour.plotting.plot_image(
    colour.cctf_encoding(demosaicing_CFA_Bayer_bilinear(CFA, 'GBRG')));


colour.plotting.plot_image(
    colour.cctf_encoding(demosaicing_CFA_Bayer_Malvar2004(CFA, 'GBRG')));

colour.plotting.plot_image(
    colour.cctf_encoding(demosaicing_CFA_Bayer_Menon2007(CFA, 'GBRG')));

height, width = CFA.shape
print(height)
cv2_imshow(CFA)

### 3.1 Using Opencv

In [ ]:
# image = cv2.imread('CFA_stream.png')
# image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_gray = cv2.imread('CFA_stream.png',cv2.IMREAD_GRAYSCALE)

image_demosaiced = cv2.cvtColor(image_gray, cv2.COLOR_BayerGR2RGB) # COLOR_BayerGB2RGB
image_rgb = image_demosaiced
image_cv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

cv2_imshow(image_cv)

## **Step 4**

In [ ]:
img_rgb = image_rgb.astype(np.float32) / 255.0 # Η colour διαβαζει το rgb οχι το bgr image

image_xyz = colour.models.sRGB_to_XYZ(img_rgb)

# image_xyz = colour.models.sRGB_to_XYZ(image_rgb)

colour.plotting.plot_image(image_xyz);
print(np.max(image_xyz))

In [ ]:
image_xyz_cv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2XYZ)
cv2_imshow(image_xyz_cv) # in GBR

## **Step 5**

In [ ]:
# cv2_imshow(image_rgb)

mean_rgb = np.mean(image_cv, axis=(0, 1))
print("Μέσο RGB:", mean_rgb)

# mask_r = cv2.inRange(image_cv[:,:,0],0,20)
# mask_g = cv2.inRange(image_cv[:,:,1],0,70)
# mask_b = cv2.inRange(image_cv[:,:,2],0,130)
# cv2_imshow(mask_b)
# mask_r = cv2.bitwise_not(mask_r)
# mask_g = cv2.bitwise_not(mask_r)
# mask_b = cv2.bitwise_not(mask_r)

# result = cv2.bitwise_and(image_cv,image_cv,mask = mask_r & mask_g & mask_b) # ???????????????????????????????????????????
# cv2_imshow(result)


min_rgb = np.array([0, 0, 0])
max_rgb = np.array([255, 70, 130])
mask = cv2.inRange(image_cv, min_rgb, max_rgb)
inverted_mask = cv2.bitwise_not(mask)
cv2_imshow(inverted_mask)

image_rgb_masked = cv2.bitwise_and(image_cv, image_cv, mask=inverted_mask)
cv2_imshow(image_rgb_masked)

## **Step 6**

In [ ]:
sample_xyz = cv2.cvtColor(image_rgb_masked, cv2.COLOR_RGB2XYZ)

Xs = sample_xyz[:, :, 0]
Ys = sample_xyz[:, :, 1]
Zs = sample_xyz[:, :, 2]

s_image = Xs + Ys + Zs
with np.errstate(divide='ignore', invalid='ignore'):
    x_image = Xs / s_image
    y_image = Ys / s_image

x_img = []
y_img = []

# mask = ~np.isnan(x_image) & ~np.isnan(y_image)
# x_img = x_image[mask]
# y_img = y_image[mask]

print(x_img)
# print(y_img)
# for T, (X, Y, Z) in zip(temps, zip(Χs, Ys, Zs)):
# for x, y, x_img, y_img in zip(curves_x, curves_y, x_image, y_image):
#     if not (np.isnan(x) or np.isnan(y)):
#        x = round(1000 * x)
#        y = round(1000 * y)


## **Step 7**

### 7.1 Histogram

In [ ]:
print(Image_x_y)

temperature_values = [image_result[y][x][0] for x, y in Image_x_y]  # Red channel

# Plot histogram
plt.hist(temperature_values, bins=20, color='orange', edgecolor='black')
plt.xlabel('Temperature Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

### 7.1 Heatmap

In [ ]:
height, width, channel = image_result.shape

heatmap = np.zeros((height, width), dtype=np.float32)

for x, y in Image_x_y:
    if 0 <= x < width and 0 <= y < height:
        heatmap[y, x] += 1  # Note: y first because of row-major order

heatmap_blur = cv2.GaussianBlur(image_result,(3,3), 11)
colored_heatmap = cv2.applyColorMap(np.uint8(255 * heatmap_blur), cv2.COLORMAP_JET)

plt.imshow(colored_heatmap)
plt.axis('off')
plt.show()

In [ ]:
plt.imshow(colored_heatmap)
plt.imshow(colored_heatmap, cmap='jet', origin='upper')  # bottom-left
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
image = cv2.resize(image_result, (width, height))

overlay = cv2.addWeighted(image, 0.6, colored_heatmap, 0.4, 0)

plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
blur = cv2.GaussianBlur(image_rgb,(3,3), 11)
cv2_imshow(blur)

In [ ]:
heatmap_img = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
cv2_imshow(heatmap_img)

In [ ]:
super_imposed_img = cv2.addWeighted(heatmap_img, 0.5, image_rgb, 0.5, 0)
cv2_imshow(super_imposed_img)